In [151]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import seaborn as sns
from collections import defaultdict
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn import datasets, linear_model
from sklearn import neighbors
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

In [221]:
## Read the Key, logerror, transaction date set
df_parcel = pd.read_csv("c:/Users/ibipul/Documents/Python Scripts/datasets/zillow_data/train_2016_v2.csv")

In [222]:
## Read the attribute set
df_properties = pd.read_csv("c:/Users/ibipul/Documents/Python Scripts/datasets/zillow_data/properties_2016.csv",low_memory=False)

In [251]:
#Checking for missing data
#for i in df_properties.columns:
#    print(i, ", Number of Non-Null Values is: ", sum(df_properties[i].notnull()*1))

In [225]:
# Categorical variables are Numeric here. Replace NaN's with 0.0, then hot one encode.
categorical_variables=['airconditioningtypeid','buildingqualitytypeid','heatingorsystemtypeid']
## Impute the NaN's in the following columns with median values
numeric_variables = ['bathroomcnt','bedroomcnt','roomcnt','fullbathcnt', 'calculatedbathnbr','unitcnt',
          'calculatedfinishedsquarefeet', 'finishedsquarefeet12','garagetotalsqft','lotsizesquarefeet','yearbuilt',
          'structuretaxvaluedollarcnt','taxvaluedollarcnt', 'landtaxvaluedollarcnt']
# This is the key variable that will help with the join
key_variable = ['parcelid']
# Full variable list:
properties_variables = key_variable + numeric_variables + categorical_variables

In [226]:
## Sub setting the properties variables
df = df_properties[properties_variables]

In [227]:
## Merging the id,logerr,transaction data set with the above
dfmain = pd.merge(df_parcel,df,on='parcelid')

In [228]:
# date parser
def get_month(transaction_date):
    s=transaction_date.split('-')
    month=int(s[1])
    return month

# Adding a new column as month of transaction
dfmain['month'] = dfmain['transactiondate'].apply(lambda x: get_month(x))

In [229]:
dfmain.shape

(90275, 21)

In [230]:
#Checking for missing data in numeric variables
for i in numeric_variables:
    print(i, sum(dfmain[i].notnull()*1))

bathroomcnt 90275
bedroomcnt 90275
roomcnt 90275
fullbathcnt 89093
calculatedbathnbr 89093
unitcnt 58353
calculatedfinishedsquarefeet 89614
finishedsquarefeet12 85596
garagetotalsqft 29937
lotsizesquarefeet 80125
yearbuilt 89519
structuretaxvaluedollarcnt 89895
taxvaluedollarcnt 90274
landtaxvaluedollarcnt 90274


In [231]:
#Creating an imputation dictionary
imputation_dict = defaultdict()
for i in numeric_variables:
    x = sum(dfmain[i].notnull()*1)
    #print(i, x)
    if (90275 -x) !=0:
        imputation_dict[i]=dfmain[i].median()

In [232]:
imputation_dict

defaultdict(None,
            {'calculatedbathnbr': 2.0,
             'calculatedfinishedsquarefeet': 1540.0,
             'finishedsquarefeet12': 1518.0,
             'fullbathcnt': 2.0,
             'garagetotalsqft': 433.0,
             'landtaxvaluedollarcnt': 192970.0,
             'lotsizesquarefeet': 7200.0,
             'structuretaxvaluedollarcnt': 132000.0,
             'taxvaluedollarcnt': 342872.0,
             'unitcnt': 1.0,
             'yearbuilt': 1970.0})

In [233]:
# Imputing missing data with median values of columns
for i in numeric_variables:
    x = sum(dfmain[i].isnull()*1)
    if x>0:
        dfmain[i].replace(np.nan,imputation_dict[i],inplace=True)
        print("imputation happened for: ", i," with ",imputation_dict[i])

imputation happened for:  fullbathcnt  with  2.0
imputation happened for:  calculatedbathnbr  with  2.0
imputation happened for:  unitcnt  with  1.0
imputation happened for:  calculatedfinishedsquarefeet  with  1540.0
imputation happened for:  finishedsquarefeet12  with  1518.0
imputation happened for:  garagetotalsqft  with  433.0
imputation happened for:  lotsizesquarefeet  with  7200.0
imputation happened for:  yearbuilt  with  1970.0
imputation happened for:  structuretaxvaluedollarcnt  with  132000.0
imputation happened for:  taxvaluedollarcnt  with  342872.0
imputation happened for:  landtaxvaluedollarcnt  with  192970.0


In [234]:
#Normalizing The Numeric variables
for i in numeric_variables:
    mu = dfmain[i].mean()
    sig = dfmain[i].std()
    dfmain[i]=(dfmain[i]-mu)/sig

In [235]:
dfmain.head()

,parcelid,logerror,transactiondate,bathroomcnt,bedroomcnt,roomcnt,fullbathcnt,calculatedbathnbr,unitcnt,calculatedfinishedsquarefeet,...,garagetotalsqft,lotsizesquarefeet,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,airconditioningtypeid,buildingqualitytypeid,heatingorsystemtypeid,month
0,11016594,0.0276,2016-01-01,-0.278285,-0.027558,-0.524366,-0.248716,-0.314477,-0.110974,-0.094574,...,0.182241,-0.166419,-0.403360,-0.273759,-0.175716,-0.102170,1.0,4.0,2.0,1
1,14366692,-0.1684,2016-01-01,1.215335,0.837168,-0.524366,0.795989,1.231281,-0.110974,0.531391,...,0.402113,-0.200236,1.920832,0.798069,0.230423,-0.098038,NaN,NaN,NaN,1
2,12098116,-0.0040,2016-01-01,0.717462,-0.892284,-0.524366,0.795989,0.716029,-0.110974,0.481659,...,0.182241,-0.132515,-1.206262,-0.564877,-0.608716,-0.550377,1.0,4.0,2.0,1
3,12643413,0.0218,2016-01-02,-0.278285,-0.892284,-0.524366,-0.248716,-0.314477,-0.110974,-1.008116,...,0.182241,0.384846,0.779865,-0.040117,-0.383490,-0.511800,1.0,4.0,2.0,1
4,14432541,-0.0050,2016-01-02,0.219588,0.837168,2.312889,-0.248716,0.200776,-0.110974,0.553013,...,1.218781,-0.179719,0.526317,-0.049431,-0.041667,-0.033352,NaN,NaN,NaN,1


In [236]:
# Checking for NaNs in categorical numeric variables
for i in categorical_variables:
    print(i, sum(dfmain[i].notnull()*1))


airconditioningtypeid 28781
buildingqualitytypeid 57364
heatingorsystemtypeid 56080


In [237]:
## Replacing NaNs in Categorical variables with 0's
for i in categorical_variables:
    dfmain[i].replace(np.nan,0.0,inplace=True)

In [238]:
# Checking for NaNs in categorical numeric variables
for i in categorical_variables:
    print(i, sum(dfmain[i].notnull()*1))

airconditioningtypeid 90275
buildingqualitytypeid 90275
heatingorsystemtypeid 90275


In [239]:
# Month is also a categorical variable so it should be one-hot encoded
categorical_variables.append('month')

In [241]:
## One hot encoding the categorical variables
columnsToEncode = categorical_variables
# Adding dummies
for feature in columnsToEncode:
        print('One Hot Encoding: ', feature)
        dfmain = pd.concat([dfmain, pd.get_dummies(dfmain[feature],prefix=feature)], axis=1)
        del dfmain[feature]


One Hot Encoding:  airconditioningtypeid
One Hot Encoding:  buildingqualitytypeid
One Hot Encoding:  heatingorsystemtypeid
One Hot Encoding:  month


In [242]:
dfmain.head()

,parcelid,logerror,transactiondate,bathroomcnt,bedroomcnt,roomcnt,fullbathcnt,calculatedbathnbr,unitcnt,calculatedfinishedsquarefeet,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,11016594,0.0276,2016-01-01,-0.278285,-0.027558,-0.524366,-0.248716,-0.314477,-0.110974,-0.094574,...,0,0,0,0,0,0,0,0,0,0
1,14366692,-0.1684,2016-01-01,1.215335,0.837168,-0.524366,0.795989,1.231281,-0.110974,0.531391,...,0,0,0,0,0,0,0,0,0,0
2,12098116,-0.0040,2016-01-01,0.717462,-0.892284,-0.524366,0.795989,0.716029,-0.110974,0.481659,...,0,0,0,0,0,0,0,0,0,0
3,12643413,0.0218,2016-01-02,-0.278285,-0.892284,-0.524366,-0.248716,-0.314477,-0.110974,-1.008116,...,0,0,0,0,0,0,0,0,0,0
4,14432541,-0.0050,2016-01-02,0.219588,0.837168,2.312889,-0.248716,0.200776,-0.110974,0.553013,...,0,0,0,0,0,0,0,0,0,0


In [243]:
dfmain.columns

Index(['parcelid', 'logerror', 'transactiondate', 'bathroomcnt', 'bedroomcnt',
       'roomcnt', 'fullbathcnt', 'calculatedbathnbr', 'unitcnt',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12',
       'garagetotalsqft', 'lotsizesquarefeet', 'yearbuilt',
       'structuretaxvaluedollarcnt', 'taxvaluedollarcnt',
       'landtaxvaluedollarcnt', 'airconditioningtypeid_0.0',
       'airconditioningtypeid_1.0', 'airconditioningtypeid_3.0',
       'airconditioningtypeid_5.0', 'airconditioningtypeid_9.0',
       'airconditioningtypeid_11.0', 'airconditioningtypeid_13.0',
       'buildingqualitytypeid_0.0', 'buildingqualitytypeid_1.0',
       'buildingqualitytypeid_4.0', 'buildingqualitytypeid_6.0',
       'buildingqualitytypeid_7.0', 'buildingqualitytypeid_8.0',
       'buildingqualitytypeid_10.0', 'buildingqualitytypeid_11.0',
       'buildingqualitytypeid_12.0', 'heatingorsystemtypeid_0.0',
       'heatingorsystemtypeid_1.0', 'heatingorsystemtypeid_2.0',
       'heatingorsystemt

In [244]:
## Creating a test-train split
train=dfmain.sample(frac=0.75,random_state=200)
test=dfmain.drop(train.index)

In [245]:
train.shape

(67706, 58)

In [246]:
test.shape

(22569, 58)

In [247]:
# Separating predictors and response variables
all_column_names = list(train.columns)
for i in ['parcelid','logerror','transactiondate']:
    all_column_names.remove(i)
# X variables
x_vars = all_column_names
y_var = 'logerror'
# Train set
train_Y = train[y_var]
train_X = train[x_vars]
#Test set
test_Y = train[y_var]
test_X = train[x_vars]

In [248]:
y_var

'logerror'

In [249]:
#Create a K-Nearest Neighbour model
knn = neighbors.KNeighborsRegressor(n_neighbors=5)
# Train the model using the training sets & Make predictions using the testing set
logerror_prediction_y = knn.fit(train_X, train_Y).predict(test_X)

In [250]:
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(test_Y, logerror_prediction_y))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(test_Y, logerror_prediction_y))

Mean squared error: 0.02
Variance score: 0.23
